In [1]:
import pandas as pd
import numpy as np
import gc
from scipy.stats import pearsonr
from concurrent.futures import ThreadPoolExecutor
from sklearn.feature_selection import mutual_info_classif

# 读取特征和标签

In [2]:
total_factor = pd.read_pickle('/home/datamake117/data/haris/dataset_0326/total_factor.pkl')
total_factor

,date,Code,0,1,2,3,4,5,6,7,...,3045,3046,3047,3048,label,label_,qcut,weight,qid,amount
0,2021-01-05,000001,0.071664,0.685760,-0.199404,-3.417535,0.258343,-5.388288,0.653179,-3.462629,...,1.800029,0.173608,1.884261,0.436159,0.056047,0.056047,0,0,0,5.037920e+06
1,2021-01-05,000002,0.033293,0.756402,-0.061616,-0.923554,0.642608,-3.592522,0.808333,-0.903918,...,1.959329,0.148608,1.831184,0.349514,0.027921,0.027921,0,0,0,1.114534e+07
2,2021-01-05,000004,0.041933,0.832400,-0.071108,-1.812693,0.475614,-1.823890,0.606061,-1.799145,...,0.716246,1.154439,0.633216,1.046688,-0.054570,-0.054570,0,0,0,1.849760e+05
3,2021-01-05,000005,0.004317,0.958121,-0.026501,-1.706366,0.342068,-3.706094,0.241379,-1.712731,...,0.191443,1.687688,0.614557,1.668922,-0.054615,-0.054615,0,0,0,3.843000e+04
4,2021-01-05,000006,0.000206,0.971685,-0.013775,-2.612419,0.450552,-1.542106,0.620690,-2.554165,...,0.496166,1.596688,0.538494,1.605645,-0.028815,-0.028815,0,0,0,3.630000e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4623284,2025-02-14,688799,0.013516,0.000000,0.000000,-1.230162,0.000000,-1.700521,0.000000,-1.153934,...,0.000000,0.000000,0.000000,0.000000,-0.006603,-0.004346,0,0,0,0.000000e+00
4623285,2025-02-14,688800,0.381315,0.542528,-0.096741,0.441431,0.576844,1.263855,1.405405,0.448139,...,0.000000,0.000000,0.000000,0.000000,-0.007086,-0.005070,0,0,0,1.025376e+06
4623286,2025-02-14,688819,0.041408,0.942160,-0.039829,-2.833302,0.272820,-2.098373,0.617021,-2.736076,...,0.000000,0.000000,0.000000,0.000000,-0.005179,-0.002209,0,0,0,5.956757e+04
4623287,2025-02-14,688981,0.126986,0.732891,-0.078574,-2.118357,0.554371,6.505091,1.456559,-2.026053,...,0.000000,0.000000,0.000000,0.000000,-0.002473,0.001849,0,0,0,2.000000e+07


In [3]:
temp = total_factor[total_factor['date'] >= pd.to_datetime("2020-07-01")]
del total_factor
total_factor = temp
del temp
gc.collect()
total_factor

,date,Code,0,1,2,3,4,5,6,7,...,3045,3046,3047,3048,label,label_,qcut,weight,qid,amount
0,2021-01-05,000001,0.071664,0.685760,-0.199404,-3.417535,0.258343,-5.388288,0.653179,-3.462629,...,1.800029,0.173608,1.884261,0.436159,0.056047,0.056047,0,0,0,5.037920e+06
1,2021-01-05,000002,0.033293,0.756402,-0.061616,-0.923554,0.642608,-3.592522,0.808333,-0.903918,...,1.959329,0.148608,1.831184,0.349514,0.027921,0.027921,0,0,0,1.114534e+07
2,2021-01-05,000004,0.041933,0.832400,-0.071108,-1.812693,0.475614,-1.823890,0.606061,-1.799145,...,0.716246,1.154439,0.633216,1.046688,-0.054570,-0.054570,0,0,0,1.849760e+05
3,2021-01-05,000005,0.004317,0.958121,-0.026501,-1.706366,0.342068,-3.706094,0.241379,-1.712731,...,0.191443,1.687688,0.614557,1.668922,-0.054615,-0.054615,0,0,0,3.843000e+04
4,2021-01-05,000006,0.000206,0.971685,-0.013775,-2.612419,0.450552,-1.542106,0.620690,-2.554165,...,0.496166,1.596688,0.538494,1.605645,-0.028815,-0.028815,0,0,0,3.630000e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4623284,2025-02-14,688799,0.013516,0.000000,0.000000,-1.230162,0.000000,-1.700521,0.000000,-1.153934,...,0.000000,0.000000,0.000000,0.000000,-0.006603,-0.004346,0,0,0,0.000000e+00
4623285,2025-02-14,688800,0.381315,0.542528,-0.096741,0.441431,0.576844,1.263855,1.405405,0.448139,...,0.000000,0.000000,0.000000,0.000000,-0.007086,-0.005070,0,0,0,1.025376e+06
4623286,2025-02-14,688819,0.041408,0.942160,-0.039829,-2.833302,0.272820,-2.098373,0.617021,-2.736076,...,0.000000,0.000000,0.000000,0.000000,-0.005179,-0.002209,0,0,0,5.956757e+04
4623287,2025-02-14,688981,0.126986,0.732891,-0.078574,-2.118357,0.554371,6.505091,1.456559,-2.026053,...,0.000000,0.000000,0.000000,0.000000,-0.002473,0.001849,0,0,0,2.000000e+07


In [4]:
# 只保留2020-07-01至2024-12-31的数据
total_factor = total_factor.fillna(0)
total_factor.head()

,date,Code,0,1,2,3,4,5,6,7,...,3045,3046,3047,3048,label,label_,qcut,weight,qid,amount
0,2021-01-05,000001,0.071664,0.685760,-0.199404,-3.417535,0.258343,-5.388288,0.653179,-3.462629,...,1.800029,0.173608,1.884261,0.436159,0.056047,0.056047,0,0,0,5037920.0
1,2021-01-05,000002,0.033293,0.756402,-0.061616,-0.923554,0.642608,-3.592522,0.808333,-0.903918,...,1.959329,0.148608,1.831184,0.349514,0.027921,0.027921,0,0,0,11145336.0
2,2021-01-05,000004,0.041933,0.832400,-0.071108,-1.812693,0.475614,-1.823890,0.606061,-1.799145,...,0.716246,1.154439,0.633216,1.046688,-0.054570,-0.054570,0,0,0,184976.0
3,2021-01-05,000005,0.004317,0.958121,-0.026501,-1.706366,0.342068,-3.706094,0.241379,-1.712731,...,0.191443,1.687688,0.614557,1.668922,-0.054615,-0.054615,0,0,0,38430.0
4,2021-01-05,000006,0.000206,0.971685,-0.013775,-2.612419,0.450552,-1.542106,0.620690,-2.554165,...,0.496166,1.596688,0.538494,1.605645,-0.028815,-0.028815,0,0,0,36300.0


# 构造因子筛选辅助数据

In [5]:
factor_columns = [col for col in total_factor.columns if col not in ['date', 'Code', 'label', 'label_', 'week_group', 'qcut', 'weight', 'qid', 'amount']]

# 计算因子每天的ic，注释掉的部分可以任意筛掉不同收益率分位数的股票
def compute_all_corrs(df, power=1, target_col='label_'):
    df = df.dropna()
    data = df[factor_columns].to_numpy()
    target = df[target_col].to_numpy()                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
    # 用因子的几次方计算ic
    data = data ** power
    # 中心化数据
    target_centered = target - target.mean()
    data_centered = data - data.mean(axis=0)
    # 协方差
    covariances = np.mean(data_centered * target_centered[:, np.newaxis], axis=0)
    # 标准差
    std_target = target.std()
    std_features = data.std(axis=0)
    std_features[std_features==0] = 1
    # 相关系数
    correlations = covariances / (std_target * std_features)

    del df
    del target
    del data
    del target_centered
    del data_centered
    del std_target
    del std_features
    del covariances
    gc.collect()
    
    return pd.Series(correlations, index=factor_columns)

In [6]:
# 按天分组计算相关性
correlation_df1 = total_factor.groupby('date').apply(lambda x: compute_all_corrs(x, power=1)).reset_index()
correlation_df2 = total_factor.groupby('date').apply(lambda x: compute_all_corrs(x, power=2)).reset_index()
correlation_df3 = total_factor.groupby('date').apply(lambda x: compute_all_corrs(x, power=3)).reset_index()
correlation_df1.set_index('date', inplace=True)
correlation_df2.set_index('date', inplace=True)
correlation_df3.set_index('date', inplace=True)
correlation_df1 = correlation_df1.abs()
correlation_df2 = correlation_df2.abs()
correlation_df3 = correlation_df3.abs()
correlation_df = pd.DataFrame(
    np.maximum.reduce([correlation_df1.values, correlation_df2.values, correlation_df3.values]),  # 逐元素最大值
    columns=correlation_df1.columns, 
    index=correlation_df1.index
    )
correlation_df.to_pickle('/home/datamake117/data/haris/dataset_0326/corr_byday_abs.pkl')
correlation_df

/home/datamake117/.local/lib/python3.9/site-packages/numpy/core/_methods.py:118: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
/tmp/ipykernel_339857/878204525.py:12: RuntimeWarning: invalid value encountered in subtract
  data_centered = data - data.mean(axis=0)
/home/datamake117/.local/lib/python3.9/site-packages/numpy/core/_methods.py:152: RuntimeWarning: invalid value encountered in reduce
  arrmean = umr_sum(arr, axis, dtype, keepdims=True, where=where)
/home/datamake117/.local/lib/python3.9/site-packages/numpy/core/_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/home/datamake117/.local/lib/python3.9/site-packages/numpy/core/_methods.py:118: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
/tmp/ipykernel_339857/878204525.py:12: RuntimeWarning: invalid value encountered in subtract
  data_centered

,0,1,2,3,4,5,6,7,8,9,...,3039,3040,3041,3042,3043,3044,3045,3046,3047,3048
date,,,,,,,,,,,,,,,,,,,,,
2021-01-05,0.024645,0.051434,0.025597,0.037390,0.087863,0.049439,0.046015,0.035646,0.083740,0.082990,...,0.133304,0.186610,0.183638,0.235885,0.225826,0.195791,0.194654,0.154622,0.204914,0.163010
2021-01-06,0.065904,0.089229,0.044900,0.083649,0.128909,0.043433,0.010981,0.079271,0.119310,0.121333,...,0.095674,0.247925,0.279289,0.295480,0.306218,0.268487,0.268880,0.207733,0.296303,0.164442
2021-01-07,0.023207,0.047016,0.058557,0.038902,0.030926,0.062522,0.018987,0.037015,0.033115,0.029802,...,0.032040,0.187282,0.225421,0.216274,0.241440,0.203095,0.180500,0.130157,0.228518,0.090074
2021-01-08,0.065805,0.084157,0.056384,0.043273,0.014075,0.060436,0.035863,0.044216,0.014951,0.013611,...,0.037819,0.045504,0.014378,0.014635,0.030264,0.026819,0.014865,0.008789,0.035688,0.025310
2021-01-11,0.090901,0.073293,0.032149,0.098631,0.020287,0.027660,0.033938,0.094122,0.021817,0.019472,...,0.122797,0.186061,0.176395,0.211667,0.202622,0.202078,0.167751,0.180961,0.173236,0.168043
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-10,0.110200,0.098417,0.028545,0.030403,0.014685,0.010630,0.017120,0.027923,0.016841,0.018594,...,0.000000,0.000000,0.000000,0.000000,0.024927,0.000000,0.000000,0.000000,0.000000,0.000000
2025-02-11,0.116535,0.133107,0.050400,0.037854,0.023850,0.053622,0.045046,0.040933,0.029344,0.027103,...,0.000000,0.000000,0.000000,0.000000,0.163131,0.000000,0.000000,0.000000,0.000000,0.000000
2025-02-12,0.092918,0.062871,0.015839,0.025889,0.037360,0.049390,0.029046,0.026893,0.044214,0.037822,...,0.000000,0.000000,0.000000,0.000000,0.160625,0.000000,0.000000,0.000000,0.000000,0.000000
